<a href="https://colab.research.google.com/github/sugarforever/wtf-langchain/blob/main/04_Prompts/04_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 提示词

## 什么是提示词？

提示词（`Prompt`）是指向模型提供的输入。这个输入通常由多个元素构成。`LangChain` 提供了一系列的类和函数，简化构建和处理提示词的过程。
- 提示词模板（Prompt Template）：对提示词参数化，提高代码的重用性。
- 示例选择器（Example Selector）：动态选择要包含在提示词中的示例

## 提示词模板

提示词模板提供了可重用提示词的机制。用户通过传递一组参数给模板，实例化图一个提示词。一个提示模板可以包含：
1. 对语言模型的指令
2. 一组少样本示例，以帮助语言模型生成更好的回复
3. 向语言模型提出的问题

In [1]:
!pip install -q langchain==0.1.0 


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from langchain import PromptTemplate
template = """
你精通多种语言，是专业的翻译官。你负责{src_lang}到{dst_lang}的翻译工作。
"""

prompt = PromptTemplate.from_template(template)
prompt.format(src_lang="英文", dst_lang="中文")

'\n你精通多种语言，是专业的翻译官。你负责英文到中文的翻译工作。\n'

### 创建模板

`PromptTemplate` 类是 `LangChain` 提供的模版基础类，它接收两个参数：
1. `input_variables` - 输入变量
2. `template` - 模版

In [3]:
multiple_input_prompt = PromptTemplate(
    input_variables=["color", "animal"],
    template="A {color} {animal} ."
)
multiple_input_prompt.format(color="black", animal="bear")

'A black bear .'

#### 聊天提示词模板

聊天模型，比如 `OpenAI` 的GPT模型，接受一系列聊天消息作为输入，每条消息都与一个角色相关联。这个消息列表通常以一定格式串联，构成模型的输入，也就是提示词。

In [4]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

system_template="You are a professional translator that translates {src_lang} to {dst_lang}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template="{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt.format_prompt(
    src_lang="English",
    dst_lang="Chinese",
    user_input="Did you eat in this morning?"
).to_messages()

[SystemMessage(content='You are a professional translator that translates English to Chinese.', additional_kwargs={}),
 HumanMessage(content='Did you eat in this morning?', additional_kwargs={}, example=False)]

#### 样本选择器

In [13]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    # 可选的样本数据
    examples=examples,
    # 提示词模版
    example_prompt=example_prompt,
    # 格式化的样本数据的最大长度，通过get_text_length函数来衡量
    max_length=12,
    # get_text_length: ...
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

# 输入量极小，因此所有样本数据都会被选中
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: big
Output:


In [17]:
import numpy as np
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector, MaxMarginalRelevanceExampleSelector
from langchain.embeddings.openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, 
    # 使用的ebeddings
    OpenAIEmbeddings(), 
    # 向量数据库
    Chroma, 
    # 要返回的数目
    k=1
)

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)